## Importing the libraries

In [1]:
import numpy as np
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
label_encoder_X = LabelEncoder()
X[:, 2] = label_encoder_X.fit_transform(X[:, 2])

ct = ColumnTransformer(
    transformers=[("Geo", OneHotEncoder(), [1])],
    remainder="passthrough",
)
X = ct.fit_transform(X)
X = X[:, 1:]

## Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Import the keras library and pachages

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

2023-08-16 12:02:52.454850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Define a create function for model

In [13]:
def create_model():
    model = Sequential()
    model.add(InputLayer(input_shape=(11, ), name='input_layer'))
    model.add(Dense(units=6, activation='relu', name='hidden_layer_1'))
    model.add(Dense(units=1, activation='sigmoid', name='output_layer'))
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
    return model

## Using cross validation score

In [ ]:
classifier = KerasClassifier(model=create_model)
parameters = {
    'batch_size': [50, 100],
    'epochs': [10, 20],
}
grid_search = GridSearchCV(
    estimator = classifier,
    param_grid = parameters,
    scoring = 'accuracy',
    cv = 5,
    n_jobs=-1
)
grid_search = grid_search.fit(X_train, y_train)


## Best model

In [17]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(f"best params  : {best_params}")
print(f"best_accuracy: {best_accuracy}")

best params  : {'batch_size': 50, 'epochs': 20}
best_accuracy: 0.8426250000000002


## Predicting the Test set results

In [18]:
y_pred = best_model.predict(X_test)
y_pred = y_pred > 0.5
y_pred

40/40 [==============================] - 0s 4ms/step


array([False, False, False, ..., False, False, False])

## Making the Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"confusion matrix:\n {cm}")
print(f"accuracy  : {accuracy:.4f}")
print(f"recall    : {recall:.4f}")
print(f"precision : {precision:.4f}")
print(f"f1        : {f1:.4f}")

confusion matrix:
 [[1521   74]
 [ 205  200]]
accuracy  : 0.8605
recall    : 0.4938
precision : 0.7299
f1        : 0.5891
